# Video Data
Performed detailed data prepping and processing on image data

In [9]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.44.0.1-py2.py3-none-any.whl size=257484150 sha256=3b6ca7c933dc34dffae89dc701082eeae82fa169ca8872febeee2e4546bbb7f4
  Stored in directory: /root/.cache/pip/wheels/d9/9b/ca/7345b72d17e1e17da37239d70631c3214ec9e541b0c9e700e2
Successfully built h2o


## Step 1: EDA

In [1]:
import cv2

# Load the video
video_path = '/content/elefant_1280p.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
duration = frame_count / frame_rate


# Step 2: Data Processing

In [2]:
frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        resized_frame = cv2.resize(frame, (224, 224))
        frames.append(resized_frame)
    else:
        break


## Step 3: Feature Extraction

In [4]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

model = VGG16(weights='imagenet', include_top=False)

# Assuming `frames` is a list of frames from the video
features = model.predict(preprocess_input(np.array(frames)))


34/34 [==============================] - 392s 12s/step


## Step 4: Clustering

In [7]:
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest

kmeans = KMeans(n_clusters=3, random_state=0).fit(features.reshape(features.shape[0], -1))
clf = IsolationForest(contamination=0.1).fit(features.reshape(features.shape[0], -1))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Step 6: Building ML Models using AutoML


In [11]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()
features_h2o = h2o.H2OFrame(features.reshape(features.shape[0], -1))
aml = H2OAutoML(max_runtime_secs=3600)



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 mins 14 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_unknownUser_hc6o6t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.115 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
